In [1]:
!pip install geopandas
!apt install libspatialindex-dev
!pip install rtree
!pip install pygeos
import geopandas as gpd
import json
import pandas as pd
import rtree
import pygeos

     |████████████████████████████████| 1.0 MB 8.5 MB/s 
     |████████████████████████████████| 6.3 MB 33.8 MB/s 
     |████████████████████████████████| 16.7 MB 38.8 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 kB]
Fetched 555 kB in 1s (379 kB/s)
Selecting previo

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


                                  identity  \
0     7ca7995d-3c1a-4127-e08e-fb8d17485017   
1     d2a7d7b5-7a26-ef18-28e0-9f83c48cc529   
2     d101ddaa-9d04-b0ec-a7e9-90270f52ff30   
3     261a8448-ec32-4e43-0e98-48d6783258ee   
4     45402d6e-cecc-fb0b-041f-6978a2e9d496   
...                                    ...   
4298  e52ff4f3-8cc2-618e-7a67-e89d49693917   
4299  30ca6018-7854-9768-de17-88e28ed2d811   
4300  83dbf965-9e58-e369-af34-f7b3506610b0   
4301  1af9d2e8-08bd-c0db-896b-417b38a3a116   
4302  7d76f600-46c5-be66-e717-09981c7390e5   

                                  climate  \
0                         ET Polar-Tundra   
1                          EF Polar-Frost   
2                     Aw Tropical-Savanna   
3                     BSh Arid-Steppe-Hot   
4                     Aw Tropical-Savanna   
...                                   ...   
4298                      ET Polar-Tundra   
4299  Csa Temperate-Dry_Summer-Hot_Summer   
4300                  Aw Tropical-Savanna 

In [2]:
shapefile = gpd.read_file("other_climate_2007_koppen_geiger.shp", encoding="utf-8")

with open('train.json', encoding="UTF-8") as f:
  data = json.load(f)

coords = pd.DataFrame.from_dict(data, orient="index")
coords = coords.drop("focal", axis=1)
coords = coords.drop("make", axis=1)
coords = coords.drop("model", axis=1)
coords = coords.rename(columns={"lat": "Latitude", "lon": "Longitude"})

In [4]:
df = gpd.GeoDataFrame(
    coords, geometry=gpd.points_from_xy(coords.Latitude, coords.Longitude))
df = df.set_crs(shapefile.crs)
newpd = gpd.sjoin(df, shapefile)

,Latitude,Longitude,geometry,index_right,identity,climate
07Xmb4Y7pxflO4l_3NTBvQ,46.675700,32.643830,POINT (46.67570 32.64383),4016,15bfd56d-9155-410d-2d9f-3c868dce70cb,BWh Arid-Desert-Hot
-ALYJTfNiRDpBJCzx0nb6Q,45.253105,19.862649,POINT (45.25311 19.86265),4016,15bfd56d-9155-410d-2d9f-3c868dce70cb,BWh Arid-Desert-Hot
0dwADu-eAYgaPfWz_S-eIw,46.725180,32.698530,POINT (46.72518 32.69853),4016,15bfd56d-9155-410d-2d9f-3c868dce70cb,BWh Arid-Desert-Hot
0J0uKGNuyMQ3Wh9608LsJw,46.217040,33.040000,POINT (46.21704 33.04000),4016,15bfd56d-9155-410d-2d9f-3c868dce70cb,BWh Arid-Desert-Hot
14c5R7vWknmHYel9dk86QS,39.712260,26.371630,POINT (39.71226 26.37163),4016,15bfd56d-9155-410d-2d9f-3c868dce70cb,BWh Arid-Desert-Hot
...,...,...,...,...,...,...
5oEYJ5QZfe25uPXtRqO9iA,37.023510,-4.570038,POINT (37.02351 -4.57004),3132,ccffaa04-99b2-1a5b-169e-d3f1e38497ac,BWh Arid-Desert-Hot
8FLBgp8eQCUEaCFMLM2Ekw,37.016591,-4.565062,POINT (37.01659 -4.56506),3132,ccffaa04-99b2-1a5b-169e-d3f1e38497ac,BWh Arid-Desert-Hot
8EPuqppuirH34I3AfBLqjA,37.016626,-4.565583,POINT (37.01663 -4.56558),3132,ccffaa04-99b2-1a5b-169e-d3f1e38497ac,BWh Arid-Desert-Hot
Ae9ezDZ_l-eBKfc0xzR1sg,37.016608,-4.565376,POINT (37.01661 -4.56538),3132,ccffaa04-99b2-1a5b-169e-d3f1e38497ac,BWh Arid-Desert-Hot


In [31]:
newpd['climate'].value_counts()

EF Polar-Frost                                   78992
BWh Arid-Desert-Hot                              76762
BSh Arid-Steppe-Hot                              27017
Dfb Cold-Withouth_dry_season-Warm_Summer         20999
Aw Tropical-Savanna                               7588
Csa Temperate-Dry_Summer-Hot_Summer               5291
Cwa Temperate-Dry_Winter-Hot_Summer               3474
Cwb Temperate-Dry_Winter-Warm_Summer              2722
BSk Arid-Steppe-Cold                              2475
Dfc Cold-Withouth_dry_season-Cold_Summer          2001
BWk Arid-Desert-Cold                              1486
Cfa Temperate-Withouth_dry_season-Hot_Summer       974
Dfa Cold-Withouth_dry_season-Hot_Summer            732
Cfb Temperate-Withouth_dry_season-Warm_Summer      374
ET Polar-Tundra                                    241
Am Tropical-Monsoon                                141
Dsa Cold-Dry_Summer-Hot_Summer                     112
Dsb Cold-Dry_Summer-Warm_Summer                     41
Csb Temper

In [36]:
newpd.to_csv("data_climates.csv")